In [1]:
import datetime
from datetime import timedelta  
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
from tqdm import tqdm 

    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    
    
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis\jupyter_notebooks


In [20]:
data_subject = pd.read_csv(
    os.path.join(r'C:/Users/User/GitHub/WebET_Analysis', 'data', 'all_trials', 'added_var', 
                 'data_subject.csv'))

data_subject.columns

Index(['run_id', 'chinFirst', 'choiceTask_amountLeftFirst', 'browser',
       'browser_version', 'device', 'platform', 'platform_version',
       'user_agent', 'chosenAmount', 'chosenDelay', 'webcam_label',
       'webcam_fps', 'webcam_height', 'webcam_width', 'prolificID',
       'birthyear', 'gender', 'ethnic', 'sight', 'glasses', 'degree',
       'eyeshadow', 'masquara', 'eyeliner', 'browliner', 'vertPosition',
       'triedChin', 'keptHead', 'optionalNote', 'session_id', 'status',
       'started_datetime', 'completed_date_time', 'time_taken', 'age',
       'num_approvals', 'num_rejections', 'prolific_score',
       'reviewed_at_datetime', 'entered_code', 'Country of Birth',
       'Current Country of Residence', 'Employment Status', 'First Language',
       'Nationality', 'Sex', 'Student Status', 'Webcam', 'fps', 'max_trial',
       'glasses_binary', 'recorded_date', 'employment_status',
       'fullTime_binary'],
      dtype='object')

In [23]:
data_et = pd.read_csv(
    os.path.join(r'C:/Users/User/GitHub/WebET_Analysis', 'data', 'all_trials', 'added_var', 
                 'data_et.csv'))
data_trial = pd.read_csv(
    os.path.join(r'C:/Users/User/GitHub/WebET_Analysis', 'data', 'all_trials', 'added_var', 
                 'data_trial.csv'))
data_subject = pd.read_csv(
    os.path.join(r'C:/Users/User/GitHub/WebET_Analysis', 'data', 'all_trials', 'added_var', 
                 'data_subject.csv')) \
    .loc[:, 
            ['run_id', 'prolificID', 
             'birthyear', 'webcam_fps', 'webcam_label', 'recorded_date', 'max_trial',
             'chosenAmount', 'chosenDelay']] \
   .drop_duplicates() \
   .rename(columns={'chosenAmount': 'bonus_USD',
                    'chosenDelay': 'bonus_delay'})
data_subject['prolificID'] = data_subject['prolificID'].str.strip()

data_prolific_int = pd.read_csv(
    os.path.join(r'C:/Users/User/GitHub/WebET_Analysis', 'data', 'prolific', 'prolific_export_int.csv')) \
    .rename(columns={'participant_id': 'prolificID'}) 

data_prolific_us = pd.read_csv(
    os.path.join(r'C:/Users/User/GitHub/WebET_Analysis', 'data', 'prolific', 'prolific_export_us.csv')) \
    .rename(columns={'participant_id': 'prolificID'}) \

data_prolific = data_prolific_int \
    .append(data_prolific_us) \
    .merge(data_subject,
           on='prolificID', 
           how='left')

overview = pd.DataFrame(
    [
        [len(data_et)],
        [len(data_trial)],
        [len(data_subject)],
        [len(data_prolific)]
    ], 
    columns=['length'],
    index=['data_et', 
           'data_trial',
           'data_subject', 
           'data_prolific'])
print(overview)

                length
data_et        2700001
data_trial      133206
data_subject       316
data_prolific      381


# Approve subjects

In [21]:
data_et = data_et.merge(
    data_trial.loc[:, ['run_id', 'chinFirst', 'trial_index', 'trial_type', 'task_nr']], 
    on=['run_id', 'trial_index'], 
    how='left'
)

In [6]:
output = []
for subject in tqdm(data_et['run_id'].unique()):
    m_x_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].mean()
    m_x_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].mean()

    m_y_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'y'
    ].mean()
    m_y_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'y'
    ].mean()

    m_count_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].count()

    m_count_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].count()
    
    n_choseTop = sum(data_trial.loc[
        (data_trial['trial_type']=='eyetracking-choice') &
        (data_trial['run_id']==subject),
        'key_press']==38)

    output.append([subject, m_x_fix, m_y_fix, m_x_choice, m_y_choice, 
                   m_count_fix, m_count_choice, 
                   n_choseTop])

output = pd.DataFrame(output, 
                      columns=['run_id', 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 
                               'm_count_fix', 'm_count_choice',
                               'n_choseTop'])
output

data_prolific = data_prolific.merge(output, on='run_id', how='left')

 15%|████████████▌                                                                    | 39/252 [00:45<04:11,  1.18s/it]


KeyboardInterrupt: 

## These subjects await review

In [6]:
data_prolific['status'].unique()

array(['APPROVED', 'RETURNED', 'REJECTED', 'TIMED-OUT'], dtype=object)

In [7]:
awaiting_review = data_prolific.loc[
    data_prolific['status']=='AWAITING REVIEW', 
    ['run_id', 'prolificID', 'started_datetime', 'time_taken', 'max_trial',
     #'m_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 'm_count_fix', 'm_count_choice',
     #'n_choseTop'
    ]
].sort_values(by='started_datetime')
awaiting_review
# View(awaiting_review)

,run_id,prolificID,started_datetime,time_taken,trial_length


In [8]:
data_prolific.loc[
    data_prolific['status']=='AWAITING REVIEW', 
    'prolificID'].to_csv(header=None, index=False, line_terminator=', ')

''

In [9]:
if 'choice_rt' in data_prolific.columns:
    data_prolific = data_prolific.drop(columns='choice_rt')
grouped = data_trial \
    .loc[
        data_trial['trial_type']=='eyetracking-choice', 
        :] \
    .groupby(['prolificID'])['rt'].mean() \
    .reset_index() \
    .rename(columns={'rt': 'choice_rt'})

data_prolific = data_prolific.merge(
    grouped, 
    on='prolificID', 
    how='left')

data_prolific.loc[
    data_prolific['status']=='AWAITING REVIEW', 
    ['run_id', 'prolificID', 'webcam_fps', 
     'choice_rt', 'bonus_USD', 'bonus_delay']
] \
    .sort_values(by='run_id')

,run_id,prolificID,webcam_fps,choice_rt,bonus_USD,bonus_delay


# data_pay

In [35]:
data_pay = data_prolific.loc[
    data_prolific['status']=='APPROVED', 
    [
        'run_id', 'prolificID', 'max_trial', 'recorded_date', 
        'age',  'Country of Birth', 'Current Country of Residence', 'First Language',
        'Nationality', 'Sex', 
        'status', 'reviewed_at_datetime', 'Country of Birth', 'entered_code',
        'session_id', 'started_datetime', 'completed_date_time', 'time_taken',
        'bonus_USD', 'bonus_delay'
    ]
]
print(len(data_pay))

267


# Bonus payment

Reformat payments

In [38]:
data_pay['bonus_delay'] = data_pay['bonus_delay'].astype(str)
data_pay['bonus_delay'] = data_pay['bonus_delay'] \
    .replace(['Today', 'Tomorrow', '7 days', 
          '15 days', '30 days', '90 days', 
          '180 days'], 
         [0, 1, 7, 15, 30, 90, 180]) \
    .astype(float)

data_pay['bonus_USD'] = data_pay['bonus_USD'].astype(str)
data_pay['bonus_USD'] = data_pay['bonus_USD'] \
    .replace({'\$':''}, regex = True) \
    .replace('50 cent', 0.5) \
    .astype(float)

data_pay.loc[:, ['bonus_USD', 'bonus_delay']].head(5)

,bonus_USD,bonus_delay
0,4.5,7.0
3,4.5,30.0
4,4.5,30.0
5,4.0,30.0
10,5.0,30.0


## Missing values

In [57]:
# For two subjects, the bonus payment was not recorded
data_pay.loc[
    data_pay['prolificID'].isin([
        '5ef6d07be683903cd5ae171d',
        '5fea6632bf9ae4a79153efdf',
        '60186dc2cc1aa8103499603a',
        '5f4fe72e9468441227166179',
        '5ec5a64c306f255ec98d5cc1',
        '5b8969006651ea000118e42e', 
        '5fb2af792942a58ffe303948', 
        '5edc20443467e28ec4e30f93',
        '5fa1192cf99e161a5cfad1cd', 
        '5d430fdf871f1700017b2a81',
        '5c95970cd676900016e1a940',
        '5ecfc227f036c902457fc44c',
        '5ecfc227f036c902457fc44c',
        '5de80e0b521f95791f912e49',
        '5ff7baf4cbc6b34ca5779e90',
        '5f493291e3e64832df458323',
        '5dddef56946f7cd753f9e2d7',
        '5ef1980b60a4447bc1a5408c',
        '5fc8232ac3b2ae180fa1a39b',
        '5fc8232ac3b2ae180fa1a39b',
        '6019a2fa2b5f81113ba676f1',
        '5cbe04b4f429ff00159de30e',
        '5e149c03864020aa6e1cf04d'
    ]), ['bonus_USD', 'bonus_delay']] = [5, 1]

In [58]:
data_pay.loc[
    data_pay[['bonus_USD', 'bonus_delay']].isnull().any(axis=1), 
    ['run_id', 'prolificID', 'max_trial', 'started_datetime', 'reviewed_at_datetime']
].sort_values(by='started_datetime')

,run_id,prolificID,max_trial,started_datetime,reviewed_at_datetime


## Bonus in other currencies

In [59]:
data_pay['bonus_GBP'] = data_pay['bonus_USD'] * 0.75
data_pay['bonus_EUR'] = data_pay['bonus_GBP'] * 1.13

## When is the bonus due?

In [60]:
data_pay.loc[data_pay['run_id']==444, 'completed_date_time'] = \
    '2021-02-13 21:52:30.000000'
    
data_pay['completed_date']=data_pay.apply(
    lambda x: datetime.datetime.strptime(
        x['completed_date_time'], '%Y-%m-%d %H:%M:%S.%f') \
            .date(),
    axis=1)

data_pay['due_on'] = data_pay['completed_date'] + \
    data_pay['bonus_delay'].map(datetime.timedelta) 

print(len(data_pay))

267


C:\Users\User\miniconda3\lib\site-packages\pandas\core\arrays\datetimelike.py:1108: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized
  warnings.warn(


In [61]:
data_due = data_pay.loc[:, 
               [
                   'prolificID', 'run_id',
                   'Nationality', 'Current Country of Residence', 'Sex',  
                   'bonus_USD', 'bonus_GBP', 'bonus_EUR', 
                   'completed_date', 'bonus_delay', 'due_on'
               ]
              ]. sort_values(by='due_on')
data_due
# View(data_due)

,prolificID,run_id,Nationality,Current Country of Residence,Sex,bonus_USD,bonus_GBP,bonus_EUR,completed_date,bonus_delay,due_on
27,5eeaa0ffaa6af11cf32ce057,30.0,Poland,Poland,Male,3.0,2.250,2.54250,2021-01-18,0.0,2021-01-18
34,5f0cb319d29147695796a208,37.0,Italy,United Kingdom,Male,3.0,2.250,2.54250,2021-01-18,0.0,2021-01-18
33,5f8472685956c40c720d0936,36.0,Italy,Italy,Male,0.5,0.375,0.42375,2021-01-18,0.0,2021-01-18
14,5c5684ef9d244c0001b29f1e,11.0,United States,United States,Male,3.5,2.625,2.96625,2021-01-18,0.0,2021-01-18
20,5f77a902fc647a327b77004a,24.0,United Kingdom,United Kingdom,Female,3.0,2.250,2.54250,2021-01-18,0.0,2021-01-18
...,...,...,...,...,...,...,...,...,...,...,...
290,5f2cc8cebc4bd01d61151550,365.0,United States,United States,Female,4.0,3.000,3.39000,2021-02-13,180.0,2021-08-12
324,5dd4767b32288646716dc98a,396.0,United States,United States,Female,4.0,3.000,3.39000,2021-02-13,180.0,2021-08-12
322,600612a62facd195d940e24d,389.0,United States,United States,Male,4.5,3.375,3.81375,2021-02-13,180.0,2021-08-12
256,6019f33f17fdc21e027f72aa,328.0,United States,United States,Male,5.0,3.750,4.23750,2021-02-13,180.0,2021-08-12


## Bonus left to pay

In [65]:
data_pay.loc[
    data_pay['due_on']>datetime.datetime.now().date(), 
    ['prolificID', 'completed_date', 'bonus_GBP', 'due_on']]

,prolificID,completed_date,bonus_GBP,due_on
22,5cf0ccf488abf10019fb775d,2021-01-18,3.375,2021-04-18
30,5ea94df3a879f002f3e9a812,2021-01-18,3.375,2021-04-18
78,5eb9940830ab6c098bd1943d,2021-01-19,3.375,2021-04-19
86,600063f2943eab0acc812ed8,2021-01-19,3.375,2021-07-18
90,5c0e4e3d87e876000151cfec,2021-01-19,3.375,2021-04-19
...,...,...,...,...
360,5ec99c80d402d64ce088be18,2021-02-13,3.000,2021-05-14
361,5fd80a7f7100cd12eb473a9f,2021-02-13,3.000,2021-03-15
365,5d36600685d1d50001affacb,2021-02-13,3.750,2021-08-12
367,5f5ec0c28c844c784bf02089,2021-02-13,3.375,2021-05-14


In [67]:
left_bonus = data_pay.loc[
    data_pay['due_on']>datetime.datetime.now().date(), 
    'bonus_GBP'].sum()

print(f'Left bonus (GBP): {left_bonus * 1.2 + (left_bonus*0.2*0.2)}')

Left bonus (GBP): 289.695


## Check on specific subjects

In [68]:
data_pay.loc[
    data_pay['prolificID']=="5f779405dacc842fc44bc767", 
    ['completed_date_time', 'bonus_USD', 'bonus_delay', 'due_on']
]

,completed_date_time,bonus_USD,bonus_delay,due_on
45,2021-01-19 09:48:30.781000,4.0,30.0,2021-02-18


# Total costs

## Basic payment

In [22]:
data_pay['basic_GBP'] = 2.25
data_pay.loc[:, ['basic_GBP', 'bonus_GBP']].head(5)

,basic_GBP,bonus_GBP
0,2.25,3.375
1,2.25,3.375
2,2.25,3.375
3,2.25,3.000
4,2.25,3.750


## Total

In [23]:
print('N =' + str(len(data_pay)) + '\n')

reward_basic = data_pay['basic_GBP'].sum()
service_fee = reward_basic * 0.2
vat = reward_basic * 0.2*0.2
total_basic = data_pay['basic_GBP'].sum() * 1.4
total_bonus = data_pay['bonus_GBP'].sum() * 1.4
total = total_basic + total_bonus

overview = pd.DataFrame(
    [
        [reward_basic], 
        [service_fee], [vat], 
        [total_basic], [total_bonus], [total]
    ], 
    columns=['GBP'],
    index= ['reward_basic', 
        'service_fee', 'vat', 
        'total_basic', 'total_bonus', 'total'])
print(str(overview) + '\n')

print('Total in €, incl. transaction fee: ' + str(total * 1.14))

N =251

                  GBP
reward_basic   564.75
service_fee    112.95
vat             22.59
total_basic    790.65
total_bonus   1063.65
total         1854.30

Total in €, incl. transaction fee: 2113.9019999999996


## Total already paid 
Basic + bonus until today. Use that to compare with prolific bills

In [24]:
bonus_GBP_already_paid = data_pay.loc[
    data_pay['due_on']<=datetime.datetime.now().date(), 
    'bonus_GBP'].sum()

reward_basic = data_pay['basic_GBP'].sum()
service_fee = (reward_basic + bonus_GBP_already_paid) * 0.2
vat = (reward_basic + bonus_GBP_already_paid) * 0.2*0.2
total_basic = data_pay['basic_GBP'].sum() * 1.4
total_bonus = bonus_GBP_already_paid * 1.4
total = total_basic + total_bonus

overview = pd.DataFrame(
    [
        [reward_basic], 
        [service_fee], [vat], 
        [total_basic], [total_bonus], [total]
    ], 
    columns=['GBP'],
    index= ['reward_basic', 
        'service_fee', 'vat', 
        'total_basic', 'total_bonus', 'total'])
print(str(overview) + '\n')

print('Total in €, incl. transaction fee: ' + str(total * 1.14))

                   GBP
reward_basic   564.750
service_fee    194.175
vat             38.835
total_basic    790.650
total_bonus    568.575
total         1359.225

Total in €, incl. transaction fee: 1549.5164999999997


# Prognosis for the larger sample

In [25]:
def cost_prognosis(n):
    reward_basic = n * data_pay['basic_GBP'].mean()
    reward_bonus = n * data_pay['bonus_GBP'].mean()
        
    total_reward = reward_basic + reward_bonus
    service_fee  = total_reward * 0.2
    vat          = total_reward * 0.2*0.2
    total_basic  = reward_basic + reward_basic*0.2 + reward_basic*0.2*0.2
    total_bonus  = reward_bonus + reward_bonus*0.2 + reward_bonus*0.2*0.2
    total        = total_reward + service_fee + vat

    overview = pd.DataFrame(
        [
            [reward_basic], [reward_bonus], [total_reward],
            [service_fee], [vat], 
            [total_basic], [total_bonus], [total], [total * 1.14]
        ], 
        columns=[n],
        index= ['reward_basic', 'reward_bonus', 'total_reward',
            'service_fee', 'vat', 
            'total_basic', 'total_bonus', 'total GBP', 'total EUR'])
    overview = round(overview, 2)
    return(overview)

print('Budget: ' + str(round(2101*0.88, 2)) + ' GBP / ' + 
      '2101.00 EUR / ' +
      'n=' + str(int(np.floor((2101*0.88)/6.6)))
     )

print('Paid:   1700.50 GBP / ' + 
      str(round(1700.5*1.15, 2)) + ' EUR / ' + 
      'n=' + str(int(np.floor(1700.5/6.6))))
buffer_EUR = 2101-1955.57
buffer_GBP = buffer_EUR * 0.88
print('Buffer: ' + str(round(buffer_GBP, 2)) + ' /  ' + 
      str(round(buffer_EUR, 2)) + ' EUR / n=' + 
      str(int(np.floor(buffer_GBP/6.6))) + '\n')
print('Collect 207 more participants.')

pd.concat([
    cost_prognosis(1),
    cost_prognosis(50),
    cost_prognosis(257), 
    cost_prognosis(207)
], axis=1)

Budget: 1848.88 GBP / 2101.00 EUR / n=280
Paid:   1700.50 GBP / 1955.57 EUR / n=257
Buffer: 127.98 /  145.43 EUR / n=19

Collect 207 more participants.


,1,50,257,207
reward_basic,2.25,112.50,578.25,465.75
reward_bonus,3.03,151.34,777.91,626.57
total_reward,5.28,263.84,1356.16,1092.32
service_fee,1.06,52.77,271.23,218.46
vat,0.21,10.55,54.25,43.69
total_basic,2.79,139.50,717.03,577.53
total_bonus,3.75,187.67,964.61,776.94
total GBP,6.54,327.17,1681.64,1354.47
total EUR,7.46,372.97,1917.07,1544.10


# Export modified prolific data

In [26]:
data_pay.to_csv('C:/Users/User/GitHub/WebET_Analysis/prolific/data_pay.csv', index=False, header=True)

# Feedback

In [27]:
print('Success! Script ran through.')

Success! Script ran through.
